# Eat Safe, Love

## Notebook Set Up

In [1]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database
print(db.list_collection_names())

[]


In [5]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [6]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

query_20 =  establishments.find(query)
print(f"Establishments with score of 20: {establishments.count_documents({'scores.Hygiene': 20})}")


# Display the first document in the results using pprint
pprint(query_20[0])

Establishments with score of 20: 0


IndexError: no such item for Cursor instance

In [ ]:
# Convert the result to a Pandas DataFrame
query_df = pd.DataFrame(query_20)

# Display the number of rows in the DataFrame

print(f"Number of rows in the DataFrame: {query_df.shape[0]}")

# Display the first 10 rows of the DataFrame
query_df.head(10)


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
london_establishments = {"LocalAuthorityName": 
                         {"$regex": "London", "$options": "i"}, 
                         "RatingValue": {"$gte": 4}}

london_4 = establishments.find(london_establishments)


# Use count_documents to display the number of documents in the result
print(f"Number of establishments in London with RatingValue >= 4: {establishments.count_documents({'LocalAuthorityName': {'$regex': 'London', '$options': 'i'}, 'RatingValue': {'$gte': 4}})}")

# Display the first document in the results using pprint
pprint(list(london_establishments)[0])


In [ ]:
# Convert the result to a Pandas DataFrame
london_df = pd.DataFrame(list(london_establishments))

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {query_dr.shape[0]}")

# Display the first 10 rows of the DataFrame

print(london_df.head(10))


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

penang_flavours = establishments.find_one({"BusinessName": "Penang Flavours"})

degree_search = 0.01
latitude = penang_flavours["geocode"].get("latitude")
longitude = penang_flavours["geocode"].get("longitude")

if isinstance(latitude, (int, float)) and isinstance(longitude, (int, float)):
    # Proceed with the query
    degree_search = 0.01
    sort = 5
    limit = 5

    top_5 = establishments.find({
        "RatingValue": sort,
        "geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
        "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search}
    }).sort("scores.Hygiene", 1).limit(limit)

    pprint(list(top_5))  # Convert cursor to list to view results
else:
    print("Latitude or Longitude values are not properly extracted or not numbers.")


In [ ]:
# Convert result to Pandas DataFrame


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

# Print the number of documents in the result

aggregation_pipeline = [
    {"$match": {"scores.Hygiene": 0}},
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}}
]

results = list(establishments.aggregate(aggregation_pipeline))

# Print the first 10 results
pprint(results[:10])
local_authority_df = pd.DataFrame(results)
print(local_authority_df.head(10))

In [ ]:
# Convert the result to a Pandas DataFrame

local_authority_df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {local_authority_df.shape[0]}")

# Display the first 10 rows of the DataFrame
print(local_authority_df.head(10))
